In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
from imutils import object_detection 
# import non_max_suppression 
import numpy as np 
import imutils #ry tool for DIP and will let us perform different transformations from our results
import cv2 
import requests 
import time 
import argparse #read commands from our command terminal inside our script.

In [8]:
# Opencv pre-trained SVM with HOG people features 
HOGCV = cv2.HOGDescriptor()
HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [10]:
def detector(image):
    '''
    @image is a numpy array
    '''

    image = imutils.resize(image, width=min(400, image.shape[1]))
    clone = image.copy()

    (rects, weights) = HOGCV.detectMultiScale(image, winStride=(8, 8),
                                              padding=(32, 32), scale=1.05)

    # Applies non-max supression from imutils package to kick-off overlapped causing false positives or detection errors
    # boxes
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    result = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    # 
    return result

In [12]:
def localDetect(image_path):
    result = []
    image = cv2.imread(image_path)
    if len(image) <= 0:
        print("[ERROR] could not read your local image")
        return result
    print("[INFO] Detecting people")
    result = detector(image)

    # shows the result
    for (xA, yA, xB, yB) in result:
        cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)

    cv2.imshow("result", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return (result, image)

In [13]:
def cameraDetect(token, device, variable, sample_time=5):

    cap = cv2.VideoCapture(0)
    init = time.time()

    # Allowed sample time for Ubidots is 1 dot/second
    if sample_time < 1:
        sample_time = 1

    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()
        frame = imutils.resize(frame, width=min(400, frame.shape[1]))
        result = detector(frame.copy())

        # shows the result
        for (xA, yA, xB, yB) in result:
            cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
        cv2.imshow('frame', frame)

        '''
        # Sends results
        if time.time() - init >= sample_time:
            print("[INFO] Sending actual frame results")
            # Converts the image to base 64 and adds it to the context
            b64 = convert_to_base64(frame)
            context = {"image": b64}
            #sendToUbidots(token, device, variable,len(result), context=context)
            init = time.time()
        '''

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

def convert_to_base64(image):
    image = imutils.resize(image, width=400)
    img_str = cv2.imencode('.png', image)[1].tostring()
    b64 = base64.b64encode(img_str)
    return b64.decode('utf-8')

In [14]:
def detectPeople(args):
#     image_path = args["image"]
    camera = True if str(args["camera"]) == 'true' else False

    # Routine to read local image
    if image_path != None and not camera:
        print("[INFO] Image path provided, attempting to read image")
        (result, image) = localDetect(image_path)
        print("[INFO] sending results")
        # Converts the image to base 64 and adds it to the context
        b64 = convert_to_base64(image)
        context = {"image": b64}

        '''
        # Sends the result
        req = sendToUbidots(TOKEN, DEVICE, VARIABLE,
                            len(result), context=context)
        if req.status_code >= 400:
            print("[ERROR] Could not send data to Ubidots")
            return req
        '''

    # Routine to read images from webcam
    if camera:
        print("[INFO] reading camera images")
        cameraDetect(TOKEN, DEVICE, VARIABLE)

In [16]:
'''
def buildPayload(variable, value, context):
    return {variable: {"value": value, "context": context}}


def sendToUbidots(token, device, variable, value, context={}, industrial=True):
    # Builds the endpoint
    url = URL_INDUSTRIAL if industrial else URL_EDUCATIONAL
    url = "{}/api/v1.6/devices/{}".format(url, device)

    payload = buildPayload(variable, value, context)
    headers = {"X-Auth-Token": token, "Content-Type": "application/json"}

    attempts = 0
    status = 400

    while status >= 400 and attempts <= 5:
        req = requests.post(url=url, headers=headers, json=payload)
        status = req.status_code
        attempts += 1
        time.sleep(1)

return req
'''

'\ndef buildPayload(variable, value, context):\n    return {variable: {"value": value, "context": context}}\n\n\ndef sendToUbidots(token, device, variable, value, context={}, industrial=True):\n    # Builds the endpoint\n    url = URL_INDUSTRIAL if industrial else URL_EDUCATIONAL\n    url = "{}/api/v1.6/devices/{}".format(url, device)\n\n    payload = buildPayload(variable, value, context)\n    headers = {"X-Auth-Token": token, "Content-Type": "application/json"}\n\n    attempts = 0\n    status = 400\n\n    while status >= 400 and attempts <= 5:\n        req = requests.post(url=url, headers=headers, json=payload)\n        status = req.status_code\n        attempts += 1\n        time.sleep(1)\n\nreturn req\n'

In [20]:
def argsParser():
    ap = argparse.ArgumentParser()
#     ap.add_argument("-i", "--image", default=None,help="path to image test file directory")
    ap.add_argument("-c", "--camera", default=False,
                    help="Set as true if you wish to use the camera")
    args = vars(ap.parse_args())

    return args

In [21]:
def main():
    args = argsParser()
    detectPeople(args)


if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [-c CAMERA]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\rishi\AppData\Roaming\jupyter\runtime\kernel-bb9a2f2d-e7da-4c1c-b49f-05604c342d91.json


SystemExit: 2